In [6]:
import pandas as pd
import numpy as np

from csh_fantasy_bot import bot
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team

In [8]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 10
l_id = '411.l.85094'
# league_id = "403.l.18782"
manager = bot.ManagerBot(league_id=l_id, week=week_number, simulation_mode=simulation_mode)
league_scorer = manager.game_week().score_comparer
opp = manager.tm.matchup(week_number)
print(f'Opponent: {manager._get_team_name(manager.lg, opp)}')

Opponent: Andrew's Astounding Team


In [9]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
Date: 2021-12-15, in: Eeli Tolvanen(7545), out: David Perron(4265)
Date: 2021-12-16, in: Nicolas Hague(7549), out: Jordan Greenway(6792)
Date: 2021-12-18, in: Charlie Coyle(4988), out: Zach Hyman(5096)
"""

roster_changes = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.game_week(week_number).all_player_predictions)
# roster_changes = None
scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
print(scoring_result)
print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != ''):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

KeyError: '[4265] not found in axis'

In [9]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.game_week().all_player_predictions.copy()
result['GP'] = games_played
gp_table = result[result.GP.notna()].sort_values('GP', ascending=False).round(2)
gp_table[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
3985,Joe Pavelski,4.0,0.89
4930,Roman Josi,4.0,0.42
4969,Mikael Granlund,4.0,-0.37
4265,David Perron,4.0,-0.10
7143,Jordan Kyrou,4.0,-0.25
6728,Artemi Panarin,4.0,1.18
6368,Sam Reinhart,3.0,0.14
5096,Zach Hyman,3.0,0.80
5281,Tyson Barrie,3.0,0.24


In [10]:
gp_table['GP'].sum()


45.0

In [59]:
def display_sort_ftps(df, ascending=False):
    return df.sort_values('fpts', ascending=ascending)[['name','eligible_positions', 'status','fantasy_status','fpts']]

In [90]:
rest_season_predictions = manager.game_week().all_player_predictions

In [88]:
# manager.game_week().all_player_predictions.head()
my_team = rest_season_predictions[rest_season_predictions.fantasy_status == 8]
display_sort_ftps(my_team)


,name,eligible_positions,status,fantasy_status,fpts
player_id,,,,,


In [98]:

avail_d_selector = rest_season_predictions.eligible_positions.map(set(['D']).issubset) 
fas = rest_season_predictions.fantasy_status == 'FA'
fas
predicted_d = rest_season_predictions[(avail_d_selector) & (fas)]
display_sort_ftps(predicted_d)
# rest_season_predictions['fantasy_status'].value_counts()
manager.game_week().all_player_predictions.loc[[7910,7908,4686],['name', 'G','A','SOG', 'HIT']]


,name,G,A,SOG,HIT
player_id,,,,,
7910,Noah Dobson,0.10,0.37,2.40,1.52
7908,Evan Bouchard,0.10,0.25,2.00,0.75
4686,Oliver Ekman-Larsson,0.09,0.29,1.73,1.35


In [91]:
display_sort_ftps(rest_season_predictions).head(30)


,name,eligible_positions,status,fantasy_status,fpts
player_id,,,,,
6743,Connor McDavid,[C],,6,2.738990
5980,Nathan MacKinnon,[C],,5,2.167157
6369,Leon Draisaitl,"[C, LW]",,10,2.129248
6752,Mikko Rantanen,"[C, RW]",,9,1.889061
3637,Alex Ovechkin,[LW],,7,1.773450
5363,Gabriel Landeskog,"[C, LW]",,11,1.748131
7109,Auston Matthews,[C],,11,1.356745
6391,David Pastrnak,[RW],,12,1.321260
4261,Max Pacioretty,[LW],,7,1.204898
